In [1]:
import os
import sys
import pyts
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Patch

from tsx.perturbation import TimeSeriesPerturbation
from tsx.xai.lime import LIMETimeSeries, XAIModels
from tsx.xai import evaluation as eva

from data_util import *
from viz import *

import itertools
import pandas as pd
from biokit.viz import corrplot

### Objectives
1. Evaluate between different XAI models but same family (Ridge, for example)
1. Evaluate between different DL model, but same XAI model
1. Evaluate the same on Multivariate Time Series
1. (Todo) -> Evaluate the same on Univariate Time Series

### Metrics to Evaluation
1. coef correlations
1. r2-scores
    the scores when building XAI model in approximation process $f(z) ~ g(z') = w * z'$


## Prepare Data Set

In [3]:
independents = ["dew", "temp", "press", "wind_direction", "wind_speed", "snow", "rain"]
dependent = "pollution"

# Load data
df = load_data_set_bejin()
x_scaler, y_scaler = get_xy_scalers(df, independents, dependent)

# Normalize data 
df_norm = df.copy()
df_norm[independents] = x_scaler.transform(df[independents].values)
df_norm[dependent] = y_scaler.transform(df[dependent].values.reshape(-1, 1))

# Global param
n_steps = 128
window_size = 8
n_variables = len(independents)
samples_size = 100

In [4]:
# Prepare predict function
wavenet = tf.keras.models.load_model(f"{DATA_DIR}/wavenet_mts_128_1.h5")
lstm = tf.keras.models.load_model(f"{DATA_DIR}/lstm_mts_128_1.h5")

def predict_fn(z, model=lstm):
    z_reshaped = z.T.reshape(1, 128, 7)
    z_hat = model.predict(z_reshaped)
    # to avoid zero coef_ for z_hat in[0, 1]
    z_hat = y_scaler.inverse_transform(z_hat.reshape(-1, 1))  
    z_hat = z_hat.ravel()   # z_hat will arround 50 - 150
    return z_hat[0]

def lstm_fn(z):
    return predict_fn(z, model=lstm)

def wavenet_fn(z):
    return predict_fn(z, model=wavenet)

In [5]:
# Test set (random)
test_set = []
n_instances = 1000
for i in range(n_instances):
    i_df = get_instance_x(df_norm, n_steps + 1, independents + 
    [dependent])
    _x = i_df.loc[:n_steps-1, independents]
    _y = i_df[dependent]   
    test_set.append((_x.values.T, _y.to_numpy()[-1]))

In [6]:
# Prepare Params for different models
scales = ["async", "sync"]
repl_fn = ["zeros", "local_mean", "global_mean"]
model_fn = ["lstm_fn", "wavenet_fn"]

params = list(itertools.product(scales, repl_fn, model_fn))
params_df = pd.DataFrame([{"scale": s, "method": m, "model":model} for s, m, model in params])
# print(params_df)
params_df.style\
    .apply(lambda s: ['background-color: %s' % ('grey' if v else '') for v in s == "async"]) \
    .apply(lambda s: ['background-color: %s' % ('green' if v else '') for v in s == "sync"]) \
    .applymap(lambda s: 'color: %s' % ('cyan' if s == "lstm_fn" else '' )) \
    .applymap(lambda s: 'color: %s' % ('orange' if s == "wavenet_fn" else '' ))

,scale,method,model
0,async,zeros,lstm_fn
1,async,zeros,wavenet_fn
2,async,local_mean,lstm_fn
3,async,local_mean,wavenet_fn
4,async,global_mean,lstm_fn
5,async,global_mean,wavenet_fn
6,sync,zeros,lstm_fn
7,sync,zeros,wavenet_fn
8,sync,local_mean,lstm_fn
9,sync,local_mean,wavenet_fn


In [7]:
# Multinrun average explains for different xai models (# dl model)

# Todo: add this function to lime.py
#   coef_to_original = m.perturb_obj._x_masked
# x_df = get_instance_x(df_norm, n_steps, independents)
# x_arr = X[0]

# Generate Explanations over 10 instances
def get_xcoef(model, 
              instances,
              scale="sync", 
              r_fn='zeros',
              window_size=window_size, 
              sample_size=samples_size):
   
    lime_ts = LIMETimeSeries(
            scale=scale, 
            window_size=window_size,                                
            sample_size=sample_size, 
            perturb_method=r_fn
        )
    lime_ts.xai_estimator = XAIModels.Ridge
    lime_ts.explain_instances(instances, predict_fn=eval(model))

    coef = lime_ts.coef
    x_arr = instances[0]    # this to get x-coef in original format
    x_coef = lime_ts.perturb_obj._x_masked(x_arr, coef)

    return x_coef

In [8]:
# Generate explanations for each option
def generate_explanations():
    X, y_true = zip(*test_set)
    explanations = []
    for scale, method, model in params:
        sample_size = 200 if scale == 'sync' else 500
        x_coef = get_xcoef(model, X[:10], scale, method, 
                        window_size=window_size, 
                        sample_size=sample_size)
        d = {"model": model, "scale": scale, "method": method, "xcoef":x_coef}
        explanations.append(d)
    return explanations
# Skip if explanations already generated:
# explanations = generate_explanations()
# np.save('data/explanations.npy',explanations)

In [9]:
explanations = np.load('data/explanations.npy', allow_pickle=True)

In [10]:
# Functions to perturb test set
def replacements(t, method='zeros'):    
    if method == 'zeros':
        # Calculate zeros replacements
        r = np.zeros_like(t)
    if method == 'mean':
        # Calculate means replacements
        m = x_arr.mean(axis=1)
        r = np.ones_like(x_arr) * m.reshape(t.shape[0], -1)
    
    return r

def mask(t, percentile=90, axis=1, interpolation='higher'):
    # Create a mask based on 90 percentile of x_coef
    #   > 1 mean on , 0 means off (disabled)
    perc = np.percentile(t, percentile, axis=axis, interpolation=interpolation)

    # This is a mask for x based on 90 percentile of x_coef
    mask = 1 - (t >= perc.reshape(t.shape[0], -1))
    return mask

def perturb(t, m=None, r=None):
    if m is None:
        m = np.ones_like(t)
    if r is None:
        r = replacements(t, 'zeros')
    # new perturbed x 
    z = t * m + r * (1 - m)
    return z

# Test:
# Get absolute value for weight
# m = mask(np.abs(x_coef), axis=1, interpolation="higher")
# r = replacements(x_arr, method="zeros")
# z = perturb(x_arr, m, r)

1. Generate explanations per option. Each use 10 instances
1. Record the explanations to dict (Optional: store to file)
1. For each option (or XAI model), generate a perturbed test set
1. Fit to model and record mse of perturbed test to the dict
1. Repeat to save all options -> the dict
1. Repeat the same but with mask is random 
1. compare mse betweet original <-> random <-> percentile-perturbed

In [11]:
X, y_true = zip(*test_set)

def perturb_X(X, x_coef=None, method="zeros", absolute=True):
    """ x_coef: if it is None, then randome used."""
    if x_coef is not None:
        if absolute:
            x_coef = np.abs(x_coef)
        m = mask(x_coef, axis=1, interpolation="higher")
    else:
        m = np.random.randint(2, size=X[0].shape)
    def _f(x):
        r = replacements(x, method=method)
        z = perturb(x, m, r)
        return z

    Z = [_f(x_i) for x_i in X]

    return Z

# Test 
e = explanations[0]
xcoef = e["xcoef"]
X_perturbed = perturb_X(X, xcoef, method="zeros")


In [12]:
# Calculate quality measure for each explanation
from sklearn.metrics import mean_squared_error, r2_score
for e in explanations:
    # Create perturbed test set
    X_pert = perturb_X(X, x_coef=e["xcoef"], method="zeros")
    X_test = np.stack([x.T for x in X_pert])
    
    fn = eval(e["model"].replace("_fn", ""))    # remove "_fn" from lstm_fn
    y_pred = fn.predict(X_test).ravel()

    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    e.update({"mse": mse, "r2":r2})

In [13]:
# Now calculate quality measure for random
# X, y_true = zip(*test_set)
X_pert = perturb_X(X, method="zeros")   # x_coef = None ~ random masked
X_test = np.stack([x.T for x in X_pert])
for e in explanations:  
    fn = eval(e["model"].replace("_fn", ""))
    y_pred = fn.predict(X_test).ravel()
    
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    e.update({"mse_random": mse, "r2_random":r2})

In [14]:
# Now calculate quality measure for original
# X, y_true = zip(*test_set)
X_test = np.stack([x.T for x in X])
for e in explanations:  
    fn = eval(e["model"].replace("_fn", ""))
    y_pred = fn.predict(X_test).ravel()
    
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    e.update({"mse_original": mse, "r2_original":r2})

In [15]:
eva_df = pd.DataFrame(list(explanations))
eva_df = eva_df.drop("xcoef", axis=1)
eva_df["mse_diff"] = eva_df.eval("mse_original - mse")
eva_df["r2_diff"] = eva_df.eval("r2_original - r2")

The quality measure in this experiment: 
- $r^2$: the higher the better (max 1.0) and possibly negative
- mean squared error (mse): the lower the better

Columns name
- mse: is the mse of the perturbed relevance based on 90 percentile of absolute coefficient (because weights could be negative)
- r2: like mse column, but for $r^2$ score
- mse_original: the original mse of test set without perturbing
- r2_original: like mse_original
- mse_random: the mse of randomly perturbation for test set
- r2_random: like mse_random
- mse_diff = mse_original - mse
    - not like accuracy in classification, in regression, mse_original is expected to be lower than perturbed mse. hence if this metric is negative, that means the after perturbation, the error increase, then the explanation is good.
- r2_diff = r2_original - r2
    - opposite the mse_diff, this metrics show how good the metrics is. If it is postive, then the original explain better the variance in regression than when we perturbed test_set to zeros.

In [16]:
def highlight_max(s):
    is_max = (s == s.max())
    return ['background-color: yellow' if v else '' for v in is_max]

def highlight_min(s):
    is_min = s == s.min()
    return ['color: %s' % 'red' if v else 'black' for v in is_min]
    
tt = eva_df.loc[:, ["scale", "model", "method", "mse","mse_diff", "r2_diff"]]
tt.style\
    .apply(highlight_max, subset=['r2_diff'])\
    .apply(highlight_min, subset=['mse_diff'])

,scale,model,method,mse,mse_diff,r2_diff
0,async,lstm_fn,zeros,0.022326,-0.000643,0.081504
1,async,wavenet_fn,zeros,0.009315,0.002291,-0.290316
2,async,lstm_fn,local_mean,0.019943,0.001740,-0.220535
3,async,wavenet_fn,local_mean,0.013827,-0.002222,0.281590
4,async,lstm_fn,global_mean,0.025435,-0.003753,0.475666
5,async,wavenet_fn,global_mean,0.008627,0.002978,-0.377452
6,sync,lstm_fn,zeros,0.021760,-0.000078,0.009855
7,sync,wavenet_fn,zeros,0.007984,0.003621,-0.458976
8,sync,lstm_fn,local_mean,0.008642,0.013041,-1.652855
9,sync,wavenet_fn,local_mean,0.010094,0.001511,-0.191567


In [17]:
eva_df.loc[:, ["scale", "model", "method", "mse_original", "mse_random", "mse"]].style.apply(highlight_max, axis=1, subset=['mse_original', "mse_random", "mse"])\
    .apply(highlight_min, axis=1, subset=['mse_original', "mse_random", "mse"])

,scale,model,method,mse_original,mse_random,mse
0,async,lstm_fn,zeros,0.021683,0.006783,0.022326
1,async,wavenet_fn,zeros,0.011605,0.015167,0.009315
2,async,lstm_fn,local_mean,0.021683,0.006783,0.019943
3,async,wavenet_fn,local_mean,0.011605,0.015167,0.013827
4,async,lstm_fn,global_mean,0.021683,0.006783,0.025435
5,async,wavenet_fn,global_mean,0.011605,0.015167,0.008627
6,sync,lstm_fn,zeros,0.021683,0.006783,0.021760
7,sync,wavenet_fn,zeros,0.011605,0.015167,0.007984
8,sync,lstm_fn,local_mean,0.021683,0.006783,0.008642
9,sync,wavenet_fn,local_mean,0.011605,0.015167,0.010094
